### *JSON Annotation 값 -> YOLO format 변경 (실습 02)*

In [2]:
import json
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

In [6]:
#json data
json_dir = "../data/anno/"
json_paths = glob.glob(os.path.join(json_dir, "*json"))

#print(json_paths)

#label dict
label_dict = {"수각류" : 0}

for json_path in json_paths :
    
    #json 읽기
    with open(json_path, 'r', encoding='utf-8') as f :
        json_data = json.load(f)
        
    #images, annotations
    images_info = json_data['images']
    annotations_info = json_data['annotations']
    
    filename = images_info['filename']
    image_id = images_info['id']
    image_width = images_info['width']
    image_height = images_info['height']
    
    #print(f"width : {image_width}, height : {image_height}")
    
    #이미지 크기가 3024 4032정도 사이즈라서 리사이즈 필요
    #하지만 이미지 사이즈 조절시에는 포인트값에 대해서도 스케일 보정이 필요
    #1024x768 위에 이미지가 비율이 4:3
    #변경하고자하는 이미지 크기 설정
    
    new_width = 1024
    new_height = 768
    
    for ann_info in annotations_info :
        if image_id == ann_info['image_id'] :
            image_path = os.path.join("../data/anno_images/", filename)
            image = cv2.imread(image_path)
        
            #image scale
            scale_x = new_width / image.shape[1]  #x축 스케일 비율
            scale_y = new_height / image.shape[0]  #y축 스케일 비율
            
            resized_image = cv2.resize(image, (new_width, new_height))
            #4032x3024 -> 1024x768
            category_name = ann_info["category_name"]
            polygons = ann_info['polygon']
            
            #폴리건 좌표를 저장한 리스트
            points = []
            
            for polygon_info in polygons :
                x = polygon_info['x']
                y = polygon_info['y']
                
                resized_x = (x * scale_x)  #폴리건 좌표 스케일
                resized_y = (y * scale_y)  #폴리건 좌표 스케일
                #x, y는 이미지 변환 이전에 polygon 좌표이므로 스케일(비율)만큼 곱해주어 변환된 이미지에 맞춰서 좌표 생성
                
                points.append((resized_x, resized_y))
            
            cv2.polylines(resized_image,
                          [
                            np.array(points, np.int32).reshape((-1,1,2))
                          ],
                          isClosed=True,
                          color=(0,255,0),
                          thickness=2)
            
            #바운딩 박스 정보(points 변수)에서 순회하며 point에 담고 첫 번째 요소를 반환
            x_coords = [point[0] for point in points]
            #바운딩 박스 정보(points 변수)에서 순회하며 point에 담고 두 번째 요소를 반환
            y_coords = [point[1] for point in points]
            #폴리건 좌표 끝점을 기준으로 바운딩박스 만들기
            x_min = int(min(x_coords))
            y_min = int(min(y_coords))
            x_max = int(max(x_coords))
            y_max = int(max(y_coords))
            
            #print(x_min, y_min, x_max, y_max)
            
            #바운딩박스 그리기
            cv2.rectangle(resized_image, (x_min, y_min), (x_max, y_max), (0,0,255), 2)
            
            #plt.imshow(resized_image)
            #plt.show()
            
            center_x = ((x_max + x_min) / (2 * new_width))
            center_y = ((y_max + y_min) / (2 * new_height))
            yolo_w = (x_max - x_min) / new_width
            yolo_h = (y_max - y_min) / new_height
            
            #print(f"{center_x}, {center_y}, {yolo_w}, {yolo_h}")
            
            #file_name
            image_name_temp = filename.replace(".jpg", "")
            
            #label_number
            label_number = label_dict[category_name]
            
        os.makedirs("../data/yolo_label_data", exist_ok=True)
        
        with open(f"../data/yolo_label_data/{image_name_temp}.txt", 'a') as f :
            
            #a모드는 덮어쓰는게 아닌 없는부분은 추가 있는부분은 유지
            f.write(f"{label_number} {center_x} {center_y} {yolo_w} {yolo_h} \n")